# Deep Deterministic Policy Gradient for Portfolio Management

**Steps to try:**
1. We first try to overfit a 3 stocks toy case using 3 years of training data.
2. Try to generalize to other years of the same stock.
3. Get some insigts on network topology and hyperparameter tuning.
4. Increase the number of stocks. (Increase action space to more than 10.)

**Possible improvement methods:**
1. Use correlated action noise
2. Use adaptive parameter noise

**Figures to show:**
1. Training: total rewards w.r.t episode
2. How the model performs on training data
3. How the model performs on testing data

## Setup

In [ ]:
import numpy as np
from utils.data import read_stock_history, index_to_date, date_to_index
import matplotlib.pyplot as plt
# for compatible with python 3
from __future__ import print_function
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# force tensorflow to use CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
%qtconsole

In [ ]:
# read the data and choose the target stocks for training a toy example
history, abbreviation = read_stock_history(filepath='utils/datasets/stocks_history_target.h5')
history = history[:, :, :4]
target_stocks = ['AAPL', 'CMCSA', 'REGN']
training_date_start = '2012-08-13'
training_date_end = '2015-08-13'  # three years training data
training_index_start = date_to_index(training_date_start)
training_index_end = date_to_index(training_date_end)
target_history = np.empty(shape=(len(target_stocks), training_index_end - training_index_start, history.shape[2]))
for i, stock in enumerate(target_stocks):
    target_history[i] = history[abbreviation.index(stock), training_index_start:training_index_end, :]

In [ ]:
# collect testing data
testing_date_start = '2015-08-13'
testing_date_end = '2017-08-12'
testing_index_start = date_to_index(testing_date_start)
testing_index_end = date_to_index(testing_date_end)
testing_history = np.empty(shape=(len(target_stocks), testing_index_end - testing_index_start, history.shape[2]))
for i, stock in enumerate(target_stocks):
    testing_history[i] = history[abbreviation.index(stock), testing_index_start:testing_index_end, :]

In [ ]:
print(target_history.shape)
print(testing_history.shape)

In [ ]:
# visualize 3 stock open price
date_list = [index_to_date(i) for i in range(target_history.shape[1])]
x = range(target_history.shape[1])
for i in range(len(target_stocks)):
    plt.figure(i)
    plt.plot(x, target_history[i, :, 1])  # open, high, low, close = [0, 1, 2, 3]
    plt.xticks(x[::200], date_list[::200], rotation=30)
    plt.title(target_stocks[i])
    plt.show()

In [ ]:
from environment.portfolio import PortfolioEnv

In [ ]:
# instantiate environment, 3 stocks, with trading cost, window_length 50, start_date sample each time
env = PortfolioEnv(target_history, target_stocks)
from model.ddpg.ddpg import DDPG
# instantiate DDPG model
ddpg_model = DDPG(env=env)
ddpg_model.build_model(load_weights=False)

In [ ]:
# starts to train the model, hopefully it would work
ddpg_model.train(print_every_step=1, verbose=False, debug=False)

In [ ]:
ddpg_model_from_file = DDPG(env=env)
ddpg_model_from_file.build_model(load_weights=True)

In [ ]:
# evaluate the model with training data
env = PortfolioEnv(target_history, target_stocks)
observation, action = env.reset()
done = False
while not done:
    observation = observation[:, :, 3] / observation[:, :, 0]
#     print(observation)
    observation = np.expand_dims(observation, axis=-1)
    action = ddpg_model_from_file.predict(observation)
    action = np.squeeze(action, axis=0)
#     print(action)
#     input('Press any key to continue...')
    observation, _, done, _ = env.step(action)
env._render()

In [ ]:
# evaluate the model with unseen data from same stock
env = PortfolioEnv(testing_history, target_stocks, steps=650, start_idx=testing_index_start, 
                   sample_start_date='2015-10-02')
observation, action = env.reset()
done = False
for i in range(10):
    observation = observation[:, :, 3] / observation[:, :, 0] * 100
    observation = np.expand_dims(observation, axis=-1)
    action = ddpg_model_from_file.predict(observation)
    action = np.squeeze(action, axis=0)
#     print(action)
    observation, _, done, _ = env.step(action)
env._render()

In [ ]:
np.savez('temp/total_reward_state_1.npz', stat=ddpg_model.total_reward_stat)

In [ ]:
# plot episode reward and save it
total_reward_stat = np.load('temp/total_reward_state_1.npz')['stat']
x = range(len(total_reward_stat))[::20]
plt.plot(x, total_reward_stat[::20])
plt.xlabel('Episode')
plt.ylabel('Reward')

In [ ]:
# collect testing data of another 3 different companies
target_stocks = ['GOOGL', 'CSX', 'MAR']
testing_date_start = '2012-08-13'
testing_date_end = '2017-08-12'
testing_index_start = date_to_index(testing_date_start)
testing_index_end = date_to_index(testing_date_end)
testing_history = np.empty(shape=(len(target_stocks), testing_index_end - testing_index_start, history.shape[2]))
for i, stock in enumerate(target_stocks):
    testing_history[i] = history[abbreviation.index(stock), testing_index_start:testing_index_end, :]

In [ ]:
# evaluate the model with unseen data from different stocks
env = PortfolioEnv(testing_history, target_stocks, steps=1700, start_idx=testing_index_start)
observation, action = env.reset()
done = False
while not done:
    observation = observation[:, :, 3] / observation[:, :, 0]
    observation = np.expand_dims(observation, axis=-1)
    action = ddpg_model.predict(observation)
    action = np.squeeze(action, axis=0)
    observation, _, done, _ = env.step(action)
env._render()

In [ ]:
ddpg_model.actor.model.save_weights(ddpg_model.actor_path)
ddpg_model.critic.model.save_weights(ddpg_model.critic_path)
ddpg_model.actor.target_model.save_weights(ddpg_model.actor_target_path)
ddpg_model.critic.target_model.save_weights(ddpg_model.critic_target_path)

In [ ]:
layer_index = 1
# check model load
ddpg_model.actor.model.get_layer(index=layer_index).get_weights()

In [ ]:
ddpg_model_from_file.actor.model.get_layer(index=layer_index).get_weights()